# This notebook will generate the synthetic data required as an input to our Facebook/Prophet model
- Some noise will be added to the data to reflect real data!
- This notebook output will be saved in the data folder (RAW)

## Two tables will be generated:
- The first table will attach every product to a cluster of products (5 clusters A, B, C, D and E will be used):

| Product_code  |  Associated cluster |
|---|---|
| CLA01  |  A  |
| CLA02  |   E  |
| CLB01  |   A  |

The product code 3 first characters define the client : CLA : Client A , CLB : Client B etc.

- The second table is the actual Sales history table:

| Product_code  | Date  |  Quantity |
|---|---|---|
|  CLB01 | 25/07/2019  | 1,000  |
|  CLB01 | 19/07/2019  |  1,500 |
|   CLA02 | 23/07/2019  |  10,000 |

### Product table generation

#### 1. Import the famous pandas and numpy libraries

In [21]:
import pandas as pd
import numpy as np

In [55]:
#This sets the seed to always generate the same data

from numpy.random import RandomState
random_state=RandomState(9999)

In [54]:
#Input parameters

number_of_clients=5
min_number_of_products_per_client=1
max_number_of_products_per_client=6

In [56]:
number_of_products_list=random_state.randint(min_number_of_products_per_client,
                                             max_number_of_products_per_client,
                                             number_of_clients)

In [57]:
number_of_products_list

array([2, 5, 2, 4, 1])

In [41]:
def letter_range(start, stop="{", step=1):
    """Yield a range of lowercase letters.""" 
    for ord_ in range(ord(start.upper()), ord(stop.upper()), step):
        yield chr(ord_)

In [63]:
clients_list=['CL'+ l for l in letter_range(chr(97),chr(97+number_of_clients))]

In [64]:
clients_list

['CLA', 'CLB', 'CLC', 'CLD', 'CLE']